In [ ]:
from pathlib import Path
import ansys.fluent.core as pyfluent
import os
import psutil

In [ ]:
session = pyfluent.launch_fluent(mode="meshing",processor_count=2, precision="double")


In [ ]:
workflow =  session.workflow
workflow.InitializeWorkflow(WorkflowType="Watertight Geometry")

In [ ]:
from inputs import *

In [ ]:
workflow.TaskObject['Import Geometry'].Arguments = {
            'FileFormat': 'CAD', #[CAD, Mesh]
            'ImportType': 'Single File', # ['Single File', 'Multiple Files']
            'LengthUnit': 'mm', # ['mm', 'm', 'cm', 'in', 'ft', 'um', 'nm'] 
            'UseBodyLabels': 'No', # ['No', 'Yes']
            'FileName': GEOM_FILE_PATH
        }

In [ ]:
print(FACESIZES)

In [ ]:
add_local_sizing = workflow.TaskObject["Add Local Sizing"]

for facesize in FACESIZES:
    add_local_sizing.Arguments = dict(
        {
            "AddChild": "yes",
            "SizeControlType": "FaceSize",
            "Name": facesize["Name"],
            "GrowthRate": facesize["GrowthRate"],
            "TargetMeshSize":facesize["TargetMeshSize"], #mm
            "FaceLabelList": facesize["FaceLabelList"],
        }
        )
    try:
        add_local_sizing.Execute()
        print(f"local sizing added.\nFacesize: {facesize}")
    except:
        print("error")


In [ ]:
generate_surface_mesh = workflow.TaskObject["Generate the Surface Mesh"]
generate_surface_mesh.Arguments = dict(
    {
        "CFDSurfaceMeshControls": {
            "CurvatureNormalAngle": SURFACE_MESH_PARAMS["CurvatureNormalAngle"], 
            "GrowthRate": SURFACE_MESH_PARAMS["GrowthRate"],
            "MaxSize": SURFACE_MESH_PARAMS["MaxSize"],
            "MinSize": SURFACE_MESH_PARAMS[ "MinSize"],
            "SizeFunctions": SURFACE_MESH_PARAMS["SizeFunctions"],
        }
    }
)
generate_surface_mesh.Execute()

generate_surface_mesh.InsertNextTask(CommandName="ImproveSurfaceMesh")
improve_surface_mesh = workflow.TaskObject["Improve Surface Mesh"]
improve_surface_mesh.Arguments.update_dict({"FaceQualityLimit": MESH_FACE_QUALITY_LIMIT})
improve_surface_mesh.Execute()


In [ ]:
describeTask=workflow.TaskObject["Describe Geometry"]
describeTask.Arguments = dict(
    CappingRequired="No",
    SetupType="The geometry consists of only fluid regions with no voids",
    ShareTopology="Yes"
)
describeTask.Execute()

In [ ]:
describeTask=workflow.TaskObject["Describe Geometry"]
print(describeTask.TaskList)

In [ ]:
workflow.TaskObject["Create Regions"].Execute()


In [ ]:
updateBoundaries = workflow.TaskObject["Update Boundaries"]


In [ ]:
updateRegions = workflow.TaskObject["Update Regions"]